# Compact Memory User Guide

Welcome to the `compact-memory` user guide! This notebook provides a hands-on introduction to the `compact-memory` library, focusing on its command-line interface.

`compact-memory` helps you manage and compress text data (like dialogue history or large documents) to fit within the limited context windows of Large Language Models (LLMs) while retaining essential information. This guide focuses on using the `compact-memory` library.

This notebook will walk you through:
1.  **Setup**: Cloning the `compact-memory` repository, changing into its directory, and installing the package along with dependencies.
2.  **Command Line Interface (CLI)**: A tour of the `compact-memory` CLI for compressing text, trying different strategies, and interacting with an LLM.
3.  **Conclusion**: Summary and next steps for using `compact-memory`.

**Important Note on Notebook Execution**: This notebook uses `%cd` to change the current directory. Subsequent cells will operate from within the cloned `compact-memory` repository root.

Let's get started!

## 1. Initial Setup: Install Dependencies

First, we need to set up the environment. This involves cloning the `compact-memory` repository, changing our current directory into it, installing the package from local source, and then downloading necessary dependencies and models.

**Note**: If you are running this notebook locally and have already cloned this repository and set up a virtual environment from within the repo root, you might be able to skip some of these steps. However, these cells are designed to work in a fresh Google Colab environment.

### 1.1 Clone `compact-memory` Repository and Change Directory

In [ ]:
# Clone the compact-memory repository
!git clone https://github.com/scottfalconer/compact-memory.git

# Change directory into the cloned repository
# Subsequent commands will run from the root of the 'compact-memory' repository
%cd compact-memory

Now that we are inside the `compact-memory` directory, we can install the package and its dependencies using relative paths.

### 1.2 Install `compact-memory` Package from Local Source

In [ ]:
# Install the compact-memory package from the local source (current directory)
# The --no-build-isolation flag can be helpful if there are issues with 
# the build environment or specific package versions.
!pip install . --no-build-isolation

### 1.4 Download spaCy Model

Download the English language model from spaCy, which is used for text processing tasks like sentence segmentation. SpaCy should have been installed as part of the `requirements.txt`.

In [ ]:
!python -m spacy download en_core_web_sm

### 1.5 Set up PYTHONPATH (Usually Not Needed with `pip install .`)

When a package is installed using `pip install .` from its root directory, it's typically placed in the Python environment's `site-packages` directory and becomes accessible. Explicitly modifying `sys.path` is usually not necessary.

The following cell is commented out.

In [ ]:
# import sys
# import os
# sys.path.append(os.getcwd()) # os.getcwd() is now the repo root, if needed.

### 1.6 Download Pre-trained Models for `compact-memory`

Download the default embedding model (for text vectorization) and a small chat model (for response generation experiments) using the `compact-memory` CLI. The CLI should be available after the `pip install .` step.

For the chat model, this guide uses `gpt2`, a smaller model suitable for local demonstrations without requiring API keys. The library can be configured to use other models, including more powerful ones like the default `openai/gpt-3.5-turbo`.

In [ ]:
# Download the default sentence transformer model for embeddings
!compact-memory dev download-embedding-model --model-name all-MiniLM-L6-v2

In [ ]:
# Download a small chat model for demonstration purposes
!compact-memory dev download-chat-model --model-name gpt2

### 1.7 Configure Offline Usage (Optional)

If you have downloaded all necessary models and want to ensure the notebook runs without attempting to access Hugging Face Hub, you can set these environment variables. For this showcase, we'll leave them commented out.

In [ ]:
# For offline use after all models are downloaded, uncomment the following lines:
# import os
# os.environ['HF_HUB_OFFLINE'] = '1'
# os.environ['TRANSFORMERS_OFFLINE'] = '1'

## 2. Command Line Interface (CLI) Showcase

`compact-memory` also features a versatile Command Line Interface (CLI) for performing common operations without needing to write Python scripts. This is handy for quick tests, batch processing, model downloads, and direct interaction with compression strategies. The CLI became available after we installed `compact-memory` using `pip`.

Since we are operating from within the `compact-memory` repository root, file paths for the CLI should also be relative to this root.

### 2.1 Basic Help Command

To view all available CLI commands and their general options, use the `--help` flag.

In [ ]:
!compact-memory --help

### 2.2 Compression Command (`compress`)

The `compress` command lets you apply a compression engine to various forms of input: a direct line of text (using `--text`), or a single file or an entire directory (by providing the path as a positional argument). The following subsections will demonstrate these use cases.

The CLI uses strategies that are built into `compact-memory` (e.g., `pipeline`, `none`, `first_last`, `read_agent_gist`, `stopword_pruner`). For these examples, we'll use some of these built-in strategies. (Note: The 'prototype' engine was mentioned in a previous version of this guide but is not available in the current CLI; 'first_last' is used in its place in the examples below.)

In [ ]:
# Example: Compress a short text string using the 'first_last' engine (Note: 'prototype' was previously mentioned but is not available in this environment)
# This command demonstrates compressing a simple line of text. 
# We use the 'first_last' engine, fitting it within a 20-token budget.
!compact-memory compress --engine first_last --text "This is a fairly long sentence that we want to compress using the command line interface to a small number of tokens." --budget 20

# Example: Compress a text file using the 'first_last' engine (Note: 'prototype' was previously mentioned but is not available in this environment)
# This command demonstrates compressing an entire file. We use 'sample_data/moon_landing/full.txt'.
# The 'first_last' engine is employed to summarize the content within a 100-token budget.
# To compress a file, provide its path as an argument using --file. For instance:
!compact-memory compress --engine first_last --file sample_data/moon_landing/full.txt --budget 100

# Example: Compress an entire directory using the 'gist' engine
# This command demonstrates compressing all supported files within a directory.
# We use the 'sample_data/moon_landing' directory.
# The 'gist' engine will be applied to each file, and the overall output might be a concatenation or structured representation.
# The `compress` command automatically detects that this is a directory and processes its contents.


In [ ]:
# Example: Compress an entire directory using the 'first_last' engine (Note: 'prototype' was previously mentioned but is not available in this environment)
# This command demonstrates compressing all supported files within a directory.
# We use the 'sample_data/moon_landing' directory.
# The 'first_last' engine will be applied to each file.
# The `compress` command uses --dir for directory paths.
!compact-memory compress --engine first_last --dir sample_data/moon_landing --budget 200
# You might want to add --output some_directory_output.json if you want to inspect results later

### 2.3 Processing a Google Drive Folder

This section demonstrates how to mount your Google Drive in Colab, select a folder, and then recursively process all text files within that folder using `compact-memory`.

First, we need to mount your Google Drive to make its files accessible to this Colab environment. When you run the cell below, you'll be prompted to authorize Colab to access your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

After successfully mounting your Drive, you need to specify the path to the folder you want to process. 
You can find this path by navigating to your Google Drive in the file explorer panel on the left (usually under '/content/drive/MyDrive/...'). 
Copy the full path to your target folder and paste it into the `folder_path` variable in the cell below.

In [ ]:
#@title Specify Google Drive Folder Path 
folder_path = '' #@param {type:"string"}

if not folder_path: 
  print("Please enter the path to your Google Drive folder in the 'folder_path' variable.") 
elif not folder_path.startswith('/content/drive'): 
  print(f"The path '{folder_path}' does not look like a valid Google Drive path. It should start with '/content/drive'.")
else:
  print(f"Target folder path set to: {folder_path}")

In [ ]:
import os
import subprocess

# Ensure folder_path is defined (it should be from a previous cell where the user inputs it)
if 'folder_path' not in globals() or not folder_path:
    print("Error: 'folder_path' is not set. Please run the cell above to set the target folder path.")
elif not os.path.isdir(folder_path):
    print(f"Error: The path '{folder_path}' is not a valid directory or does not exist.")
else:
    print(f"Attempting to compress all supported files in directory: {folder_path}")
    
    # Construct the compact-memory command to process the entire directory
    command = [
        "compact-memory", "compress",
        "--engine", "first_last",
        "--dir", folder_path,
        "--budget", "200"
        # If you want to save the output to a single JSON file (or other structured format):
        # "--output", "/content/drive/MyDrive/compressed_directory_output.json" 
        # Note: Behavior of --output with --dir might depend on the tool's implementation 
        # (e.g., single file with all results, or one file per input file in a new directory).
        # Refer to `compact-memory compress --help` for specifics if needed.
    ]
    
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=False)
        print("--- Command Output ---")
        if result.stdout:
            print("Stdout:")
            print(result.stdout)
        if result.stderr:
            # Always print stderr, as it might contain warnings or progress info even on success
            print("Stderr:")
            print(result.stderr)
        result.check_returncode() # Raise an exception for non-zero exit codes
        print("--- Processing Complete ---")
    except subprocess.CalledProcessError as e:
        print(f"Error: Command failed with exit code {e.returncode}.")
        # stdout and stderr were already printed or available in e.stdout, e.stderr if needed again
    except FileNotFoundError:
        print("Error: `compact-memory` command not found. Make sure it's installed and in your PATH.")
        print("This can happen if you haven't run the initial setup cells in this notebook (e.g., 'pip install .').")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


#### Understanding the Script and Customization

The script above performs the following actions:
- It takes the `folder_path` you specified in the preceding cell.
- It constructs and executes a single `compact-memory compress` command using the `--dir` option. This tells `compact-memory` to find and process all supported files (typically text-based files like `.txt`, `.md`, etc.) within the given directory and its subdirectories.
- The `first_last` engine is used with a budget of `200` tokens for the content from the directory.
- The output from the command (compressed text, summaries, or errors) is printed below the cell.

**Customization Options:**
- **Engine**: Change `--engine first_last` to another engine like `--engine none` or `--engine stopword_pruner` based on your needs.
- **Budget**: Adjust `--budget 200` to a different token limit for the combined content or per-file basis if the engine/CLI supports it with `--dir`.
- **Output**: To save the compressed output, you can add the `--output` flag to the `command` list in the script (e.g., `"--output", "/content/drive/MyDrive/compressed_output.json"`). The exact format and structure of the output when using `--dir` (e.g., a single file containing all results, or multiple files) depends on the `compact-memory` tool's implementation. You may need to consult `compact-memory compress --help` or experiment to see how output is structured for directory compression.
- **File Selection**: The `--dir` option typically processes files `compact-memory` recognizes. If you need finer control over which files are included or excluded (e.g., specific glob patterns), you would need to check if the `compact-memory` CLI supports additional flags for this with `--dir` (e.g., `--glob "*.txt"` or `--exclude "*.log"`). If not, and you need very specific filtering, you might revert to a script that iterates files manually (like the previous version of this example) and calls `compact-memory compress --file` for each desired file.

**Expected Output:**
The script will print a message indicating it's attempting to process the directory. This will be followed by the standard output and standard error from the `compact-memory` command. This output might include a summary of processed files, the compressed content (or a path to it if saved to a file), and any errors or warnings encountered during processing. At the end, a message "--- Processing Complete ---" or an error summary will be shown by the Python script.

#### Choosing a Engine with `--engine`

The `--engine` option is key to controlling how `compact-memory` compresses your text. Different strategies have different behaviors. The available one-shot engines currently include `first_last`, `none`, `pipeline`, `read_agent_gist`, and `stopword_pruner`.

*   **`first_last`**: A simple engine that typically keeps the first and last parts of the text. (Used in examples above as a substitute for the previously mentioned 'prototype' engine).
*   **`none`**: Passes through the text without compression (useful for baseline or token counting).
*   **`pipeline`**: Allows chaining multiple processing steps (more advanced and requires specific configuration not covered in this basic example).
*   **`read_agent_gist`**: Available engine.
*   **`stopword_pruner`**: Removes filler words and stopwords to minimize length while preserving key content.

You can switch between them easily. For example, to process our `full.txt` using `none` (no compression) instead of `first_last` (which replaced `prototype`):

In [ ]:
# Example of using a different engine (none) on the full.txt file
# To process a file, provide its path as an argument using --file. For instance:
!compact-memory compress --engine none --file sample_data/moon_landing/full.txt --budget 100

Common options for `compact-memory compress`:
-   `--engine <name>`: ID of the compression engine (e.g., `prototype`, `pipeline`).
-   `--text "<string>"`: The text string to compress.
-   `<path_to_file_or_dir>`: (Positional) Path to the text file or directory to compress when `--text` is not used.
-   `--budget <int>`: The target token budget.
-   `--tokenizer <name>`: (Optional) Specify a tokenizer if the engine shouldn't use its default.
-   `--output <path>`: (Optional) Path to save the compressed output (e.g., as JSON). The compressed text is typically printed to the standard output if `--output` is not specified. Using `--output` is recommended for saving results, especially for larger inputs or when integrating into scripts.


### 2.5 Other Useful CLI Commands

The `compact-memory` CLI offers other utilities. Here are a few, with examples of how to get their help messages:

-   `download-model`, `download-chat-model`: We used these in the setup to fetch models (e.g., `all-MiniLM-L6-v2`, `gpt2`).
-   `engine stats`: Provides statistics about the engine store's storage.
-   `engine validate`: Validates the integrity of the engine store's storage.

In [ ]:
!compact-memory engine stats --help
!echo "\n---------------------------------------\n" # Visual separator
!compact-memory engine validate --help

## 4. Conclusion and Next Steps

This notebook has provided a user-focused introduction to the `compact-memory` library, specifically covering:
-   **Installation and Setup**: How to get the library ready for use.
-   **Command Line Interface (CLI)**: Practical examples of using the CLI to compress text, try different strategies, and provide context to an LLM with the `talk` command.

With `compact-memory`, you can effectively manage text data to fit within LLM context windows. This is particularly useful for tasks like:
-   Summarizing long documents before feeding them to an LLM.
-   Maintaining context in extended conversations with chatbots.
-   Processing large amounts of text for information retrieval when interacting with LLMs.

We encourage you to experiment with the CLI commands and strategies shown in this guide using your own data.

### Further Resources:
-   **GitHub Repository**: [https://github.com/google/compact-memory](https://github.com/google/compact-memory)
-   CLI Help: Use the help commands in your terminal.
-   **Examples Folder**: Check the `examples/` directory in the repository for any other user-oriented examples or showcases that may be added.

Happy compressing!